# **Online Retail Transactions Analysis**

## Objectives

* This notebook analyzes online retail transaction data to understand customer behavior, identify popular products, and optimize business strategies. The analysis provides actionable insights for improving sales, marketing effectiveness, and customer retention.

## Inputs

* Original Data Set -  [Online Retail Transactions Dataset from Kaggle](https://www.kaggle.com/datasets/abhishekrp1517/online-retail-transactions-dataset) 
* Transformed Data Set - <TODO>

## Outputs

* Business Performance Anaalysis 
* Customer Insights Anaalysis
* Product Performance Anaalysis
* Seasonal Patterns Anaalysis

